In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import b3d
import sys

sys.path.append(str(b3d.get_root_path()))

In [4]:
from tests.sama4d.video_to_tracks.from_initialization.keypoint_tracking_task import KeypointTrackingTask
import rerun as rr

In [5]:
from tests.sama4d.data_curation import get_loaders_for_all_curated_scenes

all_scenes = get_loaders_for_all_curated_scenes()

In [16]:
scene_spec = all_scenes[0]
task = KeypointTrackingTask(
    scene_spec["feature_track_data_loader"],
    scene_name=scene_spec["scene_name"],
    n_frames=4,
)

In [10]:
rr.init(f"task--{task.name}-3")
rr.connect("127.0.0.1:8812")
task.visualize_task()

In [11]:
task_spec = task.get_task_specification()

In [12]:
from tests.sama4d.video_to_tracks.from_initialization.solvers.particle_system_patch_tracking_solver import (
    AdamPatchTracker_UsingSingleframeParticleSystemTraces,
)

solver = AdamPatchTracker_UsingSingleframeParticleSystemTraces()
solution = solver.solve(task_spec)

In [13]:
metrics = task.score(solution)
task.visualize_solution(solution, metrics)

In [14]:
solver.visualize_solver_state(task_spec)

In [15]:
import rerun.blueprint as rrb

bp = rrb.Blueprint(
    rrb.Horizontal(
        rrb.Spatial3DView(),
        rrb.Vertical(
            rrb.Spatial2DView(
                contents=["groundtruth_solution/keypoints_2d", "solution/keypoints_2D"]
            ),
            rrb.Spatial2DView(),
        ),
    )
)
rr.send_blueprint(bp)